# Домашнее задание № 5

**Ситуация:**\
Спокойное рабочее утро. \
Налив чашку утреннего кофе вы замечаете в Outlook письмо, в котором коллеги просят помощи в решении проблемы. 

Для тестирования API нового сервиса валидации адресов им необходимо получить списки реальных адресов и геокоординат которыми пользуются клиенты компании.\
Они уже обратились за помощью к коллегам DevOps. Те выгрузили из логов сетевого балансера все, что клиенты передавали в поля, предназначенные для адресов.\
Файл durty_data.csv прилагается к письму.

От вас требуется выделить из файла только похожие на адреса и гео-координаты строки. Убрать дубликаты и кавычки, а также разложить результаты на 2 разных листа Excel файла: строковые адреса на один, а гео-координаты на другой.

Используйте любые полученные в предыдущих уроках навыки для решения поставленной задачи.
- работу с Python
- работу с Unix
- работу с БД

*В идеале оформить решение в виде .ipynb файла!* В нем **должно быть**:
1) описание алгоритма решения поставленной задачи (отдельно в начале или пошагово комментариями к коду);
2) все использованные в ходе решения скрипты/команды/запросы.

Результатом работы по вашему алгоритму должен стать xlsx файл, в котором на разных листах лежат все распознанные адреса и геокоординаты.

***Важно!***
* алгоритм решения должен быть описан настолько просто, чтобы не возникало сложностей с его воспроизведением
* использовать можно только механики/модули/методики, изученные на занятиях


#### *Решение*

In [7]:
## Решение

import pandas as pd
import re

try:
    # Откроем исходный файл durty_data.csv
    with open('durty_data.csv', encoding='utf8') as file:
         
        # Создадим два списка, в которые будем добавлять отсортированные адреса и координаты
        coordinates = []
        addresses = []

        # Для каждой строки в исходном файле выполним следующие действия:
        for s in file:
            # Очистим строки от кавычек и закодированных строк
            s = re.sub('"', '', s)
            s = re.sub(r'<.*', '', s)

            # Найдем все строки с координатами и добавим их в список координат
            coordinate = re.search(r'^\d{2,3}[.]\d+,\s?\d{2,3}[.]\d+', s)
            coordinate = coordinate[0] if coordinate else None
            coordinates.append(coordinate)
            
            # Продолжим работу с исходным файлом, чтобы в нем остались только адреса
            # Удалим из строк координаты
            s = re.sub(r'^\d{2,3}[.]\d+,?\s?\d{2,3}[.]\d+', '', s)
            # Удалим все закодированные символы
            s = re.sub(r'(.*\\{2},?\s?xD.*)|(^\\{2}.*)', '', s)
            s = re.sub(r'.*Ð.*', '', s)
            s = re.sub(r'.*Ñ.*', '', s)
            # Удалим пробелы в начале и конце строки
            s = re.sub(r'(^\s+)|(\s+$)', '', s)
            # Удалим оставшиеся одиночные символы и цифры
            s = re.sub(r'^,?\s?,?]?$', '', s)
            s = re.sub(r'^\d{2,3}$', '', s)
            
            # Добавим очищенные строки в список с адресами
            addresses.append(s)

        # Работа с адресами

        # Создадим DataFrame с адресами
        df_addresses = pd.DataFrame(addresses)
        # Удалим дубликаты
        df_addresses = df_addresses.drop_duplicates()
        # Присвоим колонке название "Адреса"
        df_addresses.columns = ['Адреса']
        # Удалим первую пустую строку, которая образовалась после удаления дубликатов
        df_addresses = df_addresses.iloc[1:]

        # Работа с координатами
        
        # Создадим DataFrame с координатами
        df_coordinates = pd.DataFrame(coordinates)
        # Удалим строки с None значениями
        df_coordinates = df_coordinates.dropna()
        # Удалим дубликаты
        df_coordinates = df_coordinates.drop_duplicates()
        # Присвоим колонке название "Координаты"
        df_coordinates.columns = ['Координаты']
        
        # Вывод результатов
        # Создадим Excel файл. На одном листе будут адреса, на втором - координаты
        with pd.ExcelWriter('results.xlsx') as writer:
            df_addresses.to_excel(writer, sheet_name='Адреса', index=False)
            df_coordinates.to_excel(writer, sheet_name='Координаты', index=False)

except FileNotFoundError:
    print('Ошибка: файл durty_data.csv не найден')

except Exception as e:
    print(f"Произошла непредвиденная ошибка: {e}")